In [1]:
!pip install --upgrade smolagents


In [2]:
import os
import gradio as gr
from smolagents import (
    Tool,
    ToolCallingAgent,
    DuckDuckGoSearchTool,
    InferenceClientModel
)

token = os.getenv("HF_TOKEN")

class SearchTool(Tool):
    name = "search_tool"
    description = "Search the web and return the most relevant info."
    inputs = {"query": {"type": "string", "description": "search query"}}
    output_type = "string"

    def __init__(self):
        super().__init__()
        self.ddg = DuckDuckGoSearchTool()  

    def forward(self, query: str):
        return self.ddg(query) 

class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides answers items to users."
    inputs = {"answers": {"type": "string", "description": "answer items seperated by \n\n"}}
    output_type = "string"

    def forward(self, answers: str):
        greeting = "Hello! I am your SmolAgent AI Jerry\n\n"
        follow_up = "\n\nCan I assist you further?"
        return greeting + answers + follow_up

class AddNumbersTool(Tool):
    name = "add_numbers"
    description = "Adds two numbers."
    inputs = {
        "a": {"type": "number", "description": "first param"},
        "b": {"type": "number", "description": "second param"},
    }
    output_type = "string"

    def forward(self, a, b):
        result = a + b
        return str(result)  

model = InferenceClientModel(
    model_id="meta-llama/Llama-3.3-70B-Instruct",
    token=token,
)

agent = ToolCallingAgent(
    model=model,
    tools=[
        AddNumbersTool(),
        SearchTool(),
        FinalAnswerTool(),
    ],
    max_steps=10,
)

def run_agent(query: str):
    try:
        result = agent.run(query)
        return result
    except Exception as e:
        return f"Error: {str(e)}"

iface = gr.Interface(
    fn=run_agent,
    inputs=gr.Textbox(lines=8, placeholder="Ask me anything..."),
    outputs=gr.Textbox(lines=8),
    title="SmolAgent",
    description=(
        "This agent uses DuckDuckGo search, adds numbers, and returns final answers"
    )
)

iface.launch(server_name="0.0.0.0", server_port=7860)

* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
